In [1]:
from glob import glob
import os
import shutil
import json
from dotenv import load_dotenv
import time
from mp_api.client import MPRester
from parquetdb import ParquetDB, config


config.logging_config.loggers.parquetdb.level='DEBUG'
config.apply()


c:\Users\lllang\miniconda3\envs\materials_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
external_dir=r'C:\Users\lllang\Desktop\Current_Projects\MatGraphDB\data\external'
materials_dir = os.path.join(external_dir,'materials_project','materials')
materials_parquetdb_dir=os.path.join(external_dir,'materials_project', 'materials_ParquetDB')

endpoint='chemenv'


In [7]:
db=ParquetDB(endpoint, dir=materials_parquetdb_dir)

2024-12-12 14:03:08 - parquetdb.core.parquetdb - INFO - dir: C:\Users\lllang\Desktop\Current_Projects\MatGraphDB\data\external\materials_project\materials_ParquetDB
2024-12-12 14:03:08 - parquetdb.core.parquetdb - INFO - load_formats: ['batches', 'table', 'dataset']


In [8]:
table=db.read(columns=['material_id','valences','wyckoff_positions', 'csm',
                           'chemsys','chemenv_iucr','chemenv_iupac',
                           'chemenv_name','chemenv_name_with_alternatives','chemenv_symbol'])
print(table.shape)

2024-12-12 14:03:11 - parquetdb.core.parquetdb - INFO - Reading data
2024-12-12 14:03:11 - parquetdb.core.parquetdb - INFO - Building filter expression
2024-12-12 14:03:11 - parquetdb.core.parquetdb - INFO - Loading data from C:\Users\lllang\Desktop\Current_Projects\MatGraphDB\data\external\materials_project\materials_ParquetDB\chemenv
2024-12-12 14:03:11 - parquetdb.core.parquetdb - INFO - Loading only columns: ['material_id', 'valences', 'wyckoff_positions', 'csm', 'chemsys', 'chemenv_iucr', 'chemenv_iupac', 'chemenv_name', 'chemenv_name_with_alternatives', 'chemenv_symbol']
2024-12-12 14:03:11 - parquetdb.core.parquetdb - INFO - Using filter: None
2024-12-12 14:03:11 - parquetdb.core.parquetdb - INFO - Loading data as a <class 'pyarrow.lib.Table'> object
2024-12-12 14:03:11 - parquetdb.core.parquetdb - INFO - Reading data passed
(92117, 10)


## Valences

In [10]:
table['valences'].combine_chunks()[:20]

[
  [],
  [],
  ...
  [
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    -2,
    -2,
    -2,
    -2
  ],
  [
    1,
    1,
    1,
    1,
    4,
    4,
    -2,
    -2,
    -2,
    -2,
    -2,
    -2
  ]
]

## Wyckoff positions

In [11]:
table['wyckoff_positions'].combine_chunks()[:20]

[
  [],
  [],
  ...
  [
    "4a",
    "4a"
  ],
  [
    "4b",
    "2a"
  ]
]

## CSM (Continuous Symmetry Measure)

In [12]:
table['csm'].combine_chunks()[:20]

[
  [],
  [],
  ...
  [
    4.9822536063958145,
    3.2182980535465475
  ],
  [
    0.1320099962747838,
    0.1634662496125414
  ]
]

# Chemsys


In [13]:
table['chemsys'].combine_chunks()[:20]


[
  "Ac",
  "Ac-Au-Eu",
  "Ac-O",
  "Ac-La",
  "Ac-La",
  "Ac-Ga-O",
  "Ac-Ga-Si",
  "Ac-Hg-Te",
  "Ac",
  "Ac-Hg-Rh",
  "Ac-Sn-Zn",
  "Ac-Dy",
  "Ac-Cd-Ge",
  "Ac-Au",
  "Ac-Ag-Pb",
  "Ac-Au-Zn",
  "Ag",
  "Ag-I-O",
  "Ag-S",
  "Ag-Ge-Se"
]

## Chemenv IUCr


In [14]:
table['chemenv_iucr'].combine_chunks()[:20]


[
  [],
  [],
  ...
  [
    "[4t]",
    "[4t]"
  ],
  [
    "[4t]",
    "[4t]"
  ]
]

# Chemenv IUPAC


In [15]:
table['chemenv_iupac'].combine_chunks()[:20]


[
  [],
  [],
  ...
  [
    "T-4",
    "T-4"
  ],
  [
    "T-4",
    "T-4"
  ]
]

## Chemenv Name


In [16]:
table['chemenv_name'].combine_chunks()[:20]

[
  [],
  [],
  ...
  [
    "Tetrahedron",
    "Tetrahedron"
  ],
  [
    "Tetrahedron",
    "Tetrahedron"
  ]
]

## Chemenv Name With Alternatives

In [17]:
table['chemenv_name_with_alternatives'].combine_chunks()[:20]

[
  [],
  [],
  ...
  [
    "Tetrahedron (also known as Triangular pyramid, Trigonal pyramid)",
    "Tetrahedron (also known as Triangular pyramid, Trigonal pyramid)"
  ],
  [
    "Tetrahedron (also known as Triangular pyramid, Trigonal pyramid)",
    "Tetrahedron (also known as Triangular pyramid, Trigonal pyramid)"
  ]
]

## Chemenv Symbol

In [18]:
table['chemenv_symbol'].combine_chunks()[:20]

[
  [],
  [],
  ...
  [
    "T:4",
    "T:4"
  ],
  [
    "T:4",
    "T:4"
  ]
]

# Analysis

In [19]:
from pyarrow import compute as pc

In [26]:
non_zero_values_mask= pc.greater(pc.list_value_length(table['chemenv_symbol'].combine_chunks()), 0)

n_valid_data=pc.sum(non_zero_values_mask)
print(f"This dataset has {n_valid_data} valid entries")

This dataset has 67096 valid entries
